# Monitoring a Multiclass Classifier Model for NLP Data with Fiddler
Unstructured data such as text are usually represented as high-dimensional vectors when processed by ML models. In this example notebook we present how [Fiddler Vector Monitoring](https://www.fiddler.ai/blog/monitoring-natural-language-processing-and-computer-vision-models-part-1) can be used to monitor NLP models using a text classification use case.

Following the steps in this notebook you can see how to onboard models that deal with unstructured text inputs. In this example, we use the 20Newsgroups dataset and train a multi-class classifier that is applied to vector embeddings of text documents. 

We monitor this model at production time and assess the performance of Fiddler's vector monitoring by manufacturing synthetc drift via sampling from specific text categories at different deployment time intervals.

---

Now we perform the following steps to demonstrate how Fiddler NLP monitoring works: 

1. Connect to Fiddler 
2. Create a Project
3. Upload Baseline Dataset
4. Add Information About the Model's Schema
5. Manufacture Synthetic Data Drift and Publish Production Events
6. Get insights

## Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import os

# 1. Connect to Fiddler

First we install and import the Fiddler Python client.

In [ ]:
!pip install -q fiddler-client
import fiddler as fdl
print(f"Running client version {fdl.__version__}")

Before you can add information about your model with Fiddler, you'll need to connect using our API client.

---

**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your organization ID
3. Your authorization token

The latter two of these can be found by pointing your browser to your Fiddler URL and navigating to the **Settings** page.

In [ ]:
URL = ''
ORG_ID = ''
AUTH_TOKEN = ''

Next we run the following code block to connect to the Fiddler API.

In [ ]:
client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN)

## 2. Create a Project

Once you connect, you can create a new project by specifying a unique project ID in the client's `create_project` function.

In [ ]:
PROJECT_ID = 'nlp_newsgroups'

if not PROJECT_ID in client.list_projects():
    print(f'Creating project: {PROJECT_ID}')
    client.create_project(PROJECT_ID)
else:
    print(f'Project: {PROJECT_ID} already exists')

## 3. Upload the Baseline Dataset

Now we retrieve the 20Newsgroup dataset. This dataset is fetched from the [scikit-learn real-world datasets](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#) and pre-processed using [this notebook](https://colab.research.google.com/github/fiddler-labs/fiddler-examples/blob/main/pre-proccessing/20newsgroups_prep_vectorization.ipynb).  For simplicity sake, we have stored it here:

In [ ]:
BASELINE_DATA_PATH = 'https://media.githubusercontent.com/media/fiddler-labs/fiddler-examples/main/quickstart/data/baseline_nlp_text_multiclassifier.csv'
baseline_df = pd.read_csv(BASELINE_DATA_PATH)
baseline_df

Now we create a [DatasetInfo](https://docs.fiddler.ai/reference/fdldatasetinfo) object to describe our baseline dataset.

In [ ]:
dataset_info = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=100)
dataset_info

Next we call the [upload_dataset()](https://docs.fiddler.ai/reference/clientupload_dataset) API to upload a baseline  to Fiddler. In addition to the baseline data, we also uploaded the whole production data framework as the 'test_data' dataset which allows us to look at the model performance metrics for unseen data.

In [ ]:
DATASET_ID = 'newsgroups_baseline'

if not DATASET_ID in client.list_datasets(project_id=PROJECT_ID):
    print(f'Upload dataset {DATASET_ID}')
    client.upload_dataset(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    dataset={'baseline':baseline_df},
    info=dataset_info
)
else:
    print(f'Dataset: {DATASET_ID} already exists in Project: {PROJECT_ID}.\n'
               'The new dataset is not uploaded. (please use a different name.)') 

# 5. Add Information About the Model's Schema

Next we should tell Fiddler a bit more about our model by creating a [model_info](https://docs.fiddler.ai/reference/fdlmodelinfo) object that specifies the model's task, inputs, outputs, and other information such as the enrichments we want performed on our model's data.

Fiddler offers a powerful set of enrichment services that we can opt in to use for to enhance how we monitor our model's performance.  In this example, we opt in to have Fiddler generate embeddings for our unstructured text.  These generated embedddings are a numerical vector that represent the content and the context of our unstructured input, "original_text".  These embeddings power Fiddler's vector monitoring capability for detecting drift.

### Opt in for Fiddler Embeddings

In addition to univariate numerical features which Fiddler monitors by default, users can define custom features and ask Fiddler to monitor them. A custom feature is specified by a group of dataset columns that need to be monitored together a a vector.

Before creating a model info object, we define a custom feature using the [fdl.Enrichment()](https://docs.fiddler.ai/reference/fdlenrichment-beta) API. When creating an enrichment, a name must be assigned to the custom feature using the `name` argument. Each enrichment appears in the monitoring tab in Fiddler UI with this assigned name. Finally, the default clustering setup can be modified by passing the number of cluster centroids to the `n_clusters` argument.

Here we define an [embedding fdl.Enrichment](https://docs.fiddler.ai/reference/embedding-enrichment-beta) and then use that embedding enrichment to create a [fdl.TextEnrichment](https://docs.fiddler.ai/reference/fdltextembedding) input that can be used to track drift and to be plotted in Fiddler's UMAP visualizations.

In [ ]:
fiddler_backend_enrichments = [
    fdl.Enrichment(
        name='Enrichment Text Embedding',
        enrichment='embedding',
        columns=['original_text'],
    ),
    fdl.TextEmbedding(
        name='Original TextEmbedding',
        source_column='original_text',
        column='Enrichment Text Embedding',
        n_clusters=6
    )
]

### Generate ModelInfo Object and Add Model 

Since this notebook demonstrates a monitoring-only use case and model predictions are already added to both baseline and production data, there is no need to access the model directly or to build a surrogate model and we use the [add_model()](https://docs.fiddler.ai/reference/clientadd_model) API. This requires passing a [model_info](https://docs.fiddler.ai/reference/fdlmodelinfo) object which conitains information about our model's task, inputs, outputs, targets and enrichments that we would like to be monitored.

In [ ]:
model_task = fdl.ModelTask.MULTICLASS_CLASSIFICATION
model_target = 'target'
model_outputs= [col for col in baseline_df.columns if col.startswith('prob_')]
model_features = ['original_text']
model_categorical_target_class_details = [col[5:] for col in model_outputs]

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    dataset_id=DATASET_ID,
    features=model_features,
    target=model_target,
    outputs=model_outputs,
    custom_features = fiddler_backend_enrichments,
    model_task=model_task,
    decision_cols=['predicted_target'],
    categorical_target_class_details=model_categorical_target_class_details,
    metadata_cols=['n_tokens', 'string_size'],
    description='A multi-class calssifier trained on NLP original text inputs.'
)
model_info

In [ ]:
MODEL_ID = 'logistic_regression_multiclassifier'

if not MODEL_ID in client.list_models(project_id=PROJECT_ID):
    client.add_model(
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        model_id=MODEL_ID,
        model_info=model_info
    )
else:
    print(f'Model: {MODEL_ID} already exists in Project: {PROJECT_ID}. Please use a different name.')

# 6. Manufacture Synthetic Data Drift and Publish Production Events

Now we publish some production events into Fiddler. We publish events in data batches and manually create data drift by sampling from particular newsgroups. This allows us to evaluate the effectiveness of Fiddler vector monitoring.

In [ ]:
PROD_EVENTS_DATA_PATH = 'https://media.githubusercontent.com/media/fiddler-labs/fiddler-examples/main/quickstart/data/production_nlp_text_multiclassifier.csv'
production_df = pd.read_csv(PROD_EVENTS_DATA_PATH)
production_df

In [ ]:
batch_size = 900 #number of events per (daily) bin
event_batches_df=[]

For sanity check, we use the baseline data as the first event batch

In [ ]:
event_batches_df.append(baseline_df)

Next sample from all categories (same as baseline)

In [ ]:
n_intervals = 6
for i in range(n_intervals):
    event_batches_df.append(production_df.sample(batch_size))

Now we generate synthetic data drift by adding event batches that are sampled from specific newsgroups

In [ ]:
T1 = ['computer','science','recreation']
T2 = ['science','religion']
T3 = ['religion']
T4 = ['computer','science','religion','forsale']
T5 = ['science','religion','forsale']
T6 = ['forsale']
synthetic_intervals = [T1,
                       T2,
                       T3,T3,T3,T3,
                       T4,T4,T4,
                       T5,T5,
                       T6,
                       ]

In [ ]:
for categories in synthetic_intervals:
    production_df_subset = production_df[production_df['target'].isin(categories)]
    event_batches_df.append(production_df_subset.sample(batch_size, replace=True))

Add more intervals sampled from all categories (no data drift)

In [ ]:
n_intervals = 6
for i in range(n_intervals):
    event_batches_df.append(production_df.sample(batch_size))

### Add Timestamp to Batches and Publish Events

In [ ]:
from datetime import datetime, date, time 
today_beginning = datetime.combine(date.today(), time()).timestamp()
daily_time_gap = 24*3600 #daily time gap in seconds

In [ ]:
#start from 29 days back
timestamp= today_beginning - 29*daily_time_gap
for event_df in event_batches_df:
    timestamp_vec = [timestamp + random.randrange(daily_time_gap) for i in range(len(event_df))]
    event_df['timestamp'] = timestamp_vec
    timestamp += daily_time_gap

Lastly, let's publish events our events with synthetic drift

In [ ]:
for event_df in event_batches_df:
    client.publish_events_batch(
        project_id=PROJECT_ID,
        model_id=MODEL_ID,
        batch_source=event_df,
        timestamp_field= 'timestamp' #comment this line if you are not adding timestamps
    )

# 7. Get insights


**You're all done!**
  
You can now head to Fiddler URL and start getting enhanced observability into your model's performance.



In particular, you can go to your model's default dashboard in Fiddler and check out the resulting drift chart . Bellow is a sceernshot of the data drift chart after running this notebook on the [Fiddler demo](https://demo.fiddler.ai/) deployment. (Annotation bubbles are not generated by the Fiddler UI.)

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/nlp_multiiclass_drift.png" />
        </td>
    </tr>
</table>